# Nepali GPT Experiment

First line of devkota's Bhikhari poem:

 हेर भिखारी अडि अडि आयो

In [11]:
text = "हेर भिखारी अडि अडि आयो"
print(text)
ltext = list(text)
len(text)
print(ltext)

हेर भिखारी अडि अडि आयो
['ह', 'े', 'र', ' ', 'भ', 'ि', 'ख', 'ा', 'र', 'ी', ' ', 'अ', 'ड', 'ि', ' ', 'अ', 'ड', 'ि', ' ', 'आ', 'य', 'ो']


If you count the number of characters in the above line of text, there are actually `15` user-perceived characters. But when we use the `len` method on that text, it returns `22`.

Non-english texts especially the ones that may make use of multiple Unicode codepoint to represent a single user-perceived character cannot be split using Python's built-in text splitting routines. They need special functions that are aware of graphemes.

Look at: https://github.com/alvinlindstam/grapheme/tree/master

Following has some good discussion on the grapheme cluster algorithm: https://bugs.python.org/issue30717

In [9]:
import grapheme

In [13]:
grapheme.length(text)
gs = grapheme.graphemes(text)
for g in gs:
    print(g)

हे
र
 
भि
खा
री
 
अ
डि
 
अ
डि
 
आ
यो


If you notice, `grapheme` module above correctly returns `15`.

Devanagari Unicode: https://unicode.org/charts/PDF/U0900.pdf

In [18]:
new_text = "ज्ञानीत्रक्ष"
new_list = list(new_text)
print(new_list)

['ज', '्', 'ञ', 'ा', 'न', 'ी', 'त', '्', 'र', 'क', '्', 'ष']


In [19]:
for gr in grapheme.graphemes(new_text):
    print(gr)

ज्
ञा
नी
त्
र
क्
ष


Graphemes works better than just using single Unicode codepoint. Even then, since some characters that are normally treated as a single unit are not defined in the standard, it can still be confusing. For now, we will work with whatever we have.

We will using grapheme as the single unit to construct the language model.

In [15]:
import os
f = open('../devkota/bhikari.txt', 'r')
txt = f.read()

In [24]:
print(txt)

(१)
हेर भिखारी अडि अडि आयो
करुण दृष्टिले नजर उठायो।
गाढा दुखको मौन प्रकाश।
झिना आशा-तार बजायो
घाम उज्यालो आँगन पास।
एक बिन्दुमा गोल खसायो
जीवनको इतिहास।
(२)
हेर, हेर ती झुत्रा चिथरा
हाय! हे समय निष्ठुर।
जीवनप थमा बिचार पथरा!
काँपिरहेको थुरथुर।
 झल्लर झोली बढाउँछ, बबुरा।
करले अस्थिर, कातर।
(३)
बर्ष-बर्षका हेर तुषारा 
शिरको उपर गिरेको।
 हेर आँसुका खहरेहरूका
 मुखमा खोंच परेको,
 दिन-दिनको त्यो छातीमाथी 
चिरा चारक्क चिरेको।
(४)
अडि अडि धरमर स्वाँ स्वाँ गर्दै 
मौन विलौना वरिपरी भर्दै 
आर्तनादका हृदय फुटी,
 जड लट्ठीको भरमा पर्दै
 भन्दछ स्वरले छाती चिर्दै 
चामल एक मुठी।
 जीवनभरको एक पुकारा 
चामल एक मुठी।
(५)
 मानिससँगमा मानिसको यो 
अन्तर्दिलको रोदन। 
भाईहरुसँग भिक्षाको यो 
मुठी द याको याचना। 
घाम उज्यालो आँगनमा यो 
एक आँध्यारो अवलोकन।
 गुलबहरूको हाँसोबिच यो 
एक उन्यौँको रोदन।
(६)
 को होला यो, कसको छोरा?
कसको बाबु गरीब? 
कुन आमाले काख लिँदामा 
बल्थे दृगका दुइ दीप?
 कुन आशाले नजर खुलायो 
सूर्य चाद्रको नजरसमिप?
 किन मुर्झायो? किन वौलायो?
 किन मधुरो यो जीवन्- व्दीप? 
(७)
बुद्ध देवको नजर अगाडि
 यही

In [25]:
lines = txt.splitlines()

In [27]:
lines[0]

'(१)'

In [28]:
for line in lines:
    print(line)

(१)
हेर भिखारी अडि अडि आयो
करुण दृष्टिले नजर उठायो।
गाढा दुखको मौन प्रकाश।
झिना आशा-तार बजायो
घाम उज्यालो आँगन पास।
एक बिन्दुमा गोल खसायो
जीवनको इतिहास।
(२)
हेर, हेर ती झुत्रा चिथरा
हाय! हे समय निष्ठुर।
जीवनप थमा बिचार पथरा!
काँपिरहेको थुरथुर।
 झल्लर झोली बढाउँछ, बबुरा।
करले अस्थिर, कातर।
(३)
बर्ष-बर्षका हेर तुषारा 
शिरको उपर गिरेको।
 हेर आँसुका खहरेहरूका
 मुखमा खोंच परेको,
 दिन-दिनको त्यो छातीमाथी 
चिरा चारक्क चिरेको।
(४)
अडि अडि धरमर स्वाँ स्वाँ गर्दै 
मौन विलौना वरिपरी भर्दै 
आर्तनादका हृदय फुटी,
 जड लट्ठीको भरमा पर्दै
 भन्दछ स्वरले छाती चिर्दै 
चामल एक मुठी।
 जीवनभरको एक पुकारा 
चामल एक मुठी।
(५)
 मानिससँगमा मानिसको यो 
अन्तर्दिलको रोदन। 
भाईहरुसँग भिक्षाको यो 
मुठी द याको याचना। 
घाम उज्यालो आँगनमा यो 
एक आँध्यारो अवलोकन।
 गुलबहरूको हाँसोबिच यो 
एक उन्यौँको रोदन।
(६)
 को होला यो, कसको छोरा?
कसको बाबु गरीब? 
कुन आमाले काख लिँदामा 
बल्थे दृगका दुइ दीप?
 कुन आशाले नजर खुलायो 
सूर्य चाद्रको नजरसमिप?
 किन मुर्झायो? किन वौलायो?
 किन मधुरो यो जीवन्- व्दीप? 
(७)
बुद्ध देवको नजर अगाडि
 यही

In [29]:
lines[1].split()

['हेर', 'भिखारी', 'अडि', 'अडि', 'आयो']

In [35]:
for word in lines[3].split():
    for g in grapheme.graphemes(word):
        print(g)

गा
ढा
दु
ख
को
मौ
न
प्
र
का
श
।


In Nepali, there are twelve vowels and 33 consonant. Each 33 consonant can then have 12 different variation. That means there are in total (12 + 33 * 12) consonant variation. Counting all the vowels, there are (12 * 33) user perceived characters.

In [50]:
num_vowels = 12
num_consonants = 33
num_variations = 12
num_digits=10
total_user_perceived_chars = num_vowels + num_digits + num_consonants + num_consonants*num_variations
print(total_user_perceived_chars)

451


In [40]:
import torch

In [41]:
t = torch.randn(5)

In [42]:
t

tensor([-0.2630, -3.0752,  0.2395, -0.2381, -1.2377])

In [1]:
nep_consonants = "क ख ग घ ङ च छ ज झ ञ ट ठ ड ढ ण त थ द ध न प फ ब भ म य र ल व श ष स ह क्ष त्र ज्ञ"
nep_variation_chars = "् ा ि ी ु ू े ै ो ौ ृ"
nep_composite_chars = "ं ँ ः"

In [2]:
nep_graphemes = nep_consonants.split()
for ng in nep_consonants.split():
    for cc in nep_composite_chars.split():
        composite_ch_var = ng+cc
        nep_graphemes.append(composite_ch_var)

In [4]:
for nc in nep_consonants.split():
    for nvc in nep_variation_chars.split():
        composite_ch = nc+nvc
        nep_graphemes.append(composite_ch)
        for cc in nep_composite_chars.split():
            composite_ch_var = composite_ch+cc
            nep_graphemes.append(composite_ch_var)

In [6]:
nep_vowels_graphemes = "अ आ इ ई उ ऊ ए ऐ ओ औ अं अः"
# 'extend' appends individual elements from one list to the other
nep_graphemes.extend(nep_vowels_graphemes.split())

nep_extendable_vowels_graphemes = "अ आ इ ई उ ऊ ए ऐ ओ औ"
nep_composite_chars_vowels = "ँ"

for ev in nep_extendable_vowels_graphemes.split():
    for nccv in nep_co

nep_digits = "० १ २ ३ ४ ५ ६ ७ ८ ९"
nep_graphemes.extend(nep_digits.split())

nep_special_graphemes = "ऋ । ( ) ? - ! : ,"
nep_graphemes.extend(nep_special_graphemes.split())
nep_graphemes.append(' ') # Add 'space' as a valid character
print(nep_graphemes)

['क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', 'क्ष', 'त्र', 'ज्ञ', 'कं', 'कँ', 'कः', 'खं', 'खँ', 'खः', 'गं', 'गँ', 'गः', 'घं', 'घँ', 'घः', 'ङं', 'ङँ', 'ङः', 'चं', 'चँ', 'चः', 'छं', 'छँ', 'छः', 'जं', 'जँ', 'जः', 'झं', 'झँ', 'झः', 'ञं', 'ञँ', 'ञः', 'टं', 'टँ', 'टः', 'ठं', 'ठँ', 'ठः', 'डं', 'डँ', 'डः', 'ढं', 'ढँ', 'ढः', 'णं', 'णँ', 'णः', 'तं', 'तँ', 'तः', 'थं', 'थँ', 'थः', 'दं', 'दँ', 'दः', 'धं', 'धँ', 'धः', 'नं', 'नँ', 'नः', 'पं', 'पँ', 'पः', 'फं', 'फँ', 'फः', 'बं', 'बँ', 'बः', 'भं', 'भँ', 'भः', 'मं', 'मँ', 'मः', 'यं', 'यँ', 'यः', 'रं', 'रँ', 'रः', 'लं', 'लँ', 'लः', 'वं', 'वँ', 'वः', 'शं', 'शँ', 'शः', 'षं', 'षँ', 'षः', 'सं', 'सँ', 'सः', 'हं', 'हँ', 'हः', 'क्षं', 'क्षँ', 'क्षः', 'त्रं', 'त्रँ', 'त्रः', 'ज्ञं', 'ज्ञँ', 'ज्ञः', 'क्', 'क्ं', 'क्ँ', 'क्ः', 'का', 'कां', 'काँ', 'काः', 'कि', 'किं', 'किँ', 'किः', 'की', 'कीं', 'कीँ', 'कीः', 'कु', 'कुं', 'कुँ', 'कुः', 'कू', 'कूं',

In [13]:
sent = "दु:खको दिलामा ईश्वर बोल्दछ"

gtoi = {s:i+1 for i, s in enumerate(nep_graphemes)} #map of all possible alphabets to index
itog = {i+1:s for i, s in enumerate(nep_graphemes)}
# for g in itog:
#    print(itog.get(g, "<INV>"))

def encode(text):
    enc = []
    for g in grapheme.graphemes(text):
        if g in gtoi:
            enc.append(gtoi.get(g, "9999"))
        else:
            print(g)
    return enc

def decode(encoding):
    str = "";
    for enc in encoding:
        str += itog.get(enc, " ")
    return str

# Making sure that the decoding the encoded sentence gives us the original text back
print(sent)
print(encode(sent))
print(decode(encode(sent)))
print(sent == (decode(encode(sent))))

दु:खको दिलामा ईश्वर बोल्दछ
[909, 1758, 2, 177, 1760, 901, 1337, 1205, 1760, 1732, 1421, 29, 27, 1760, 1145, 1333, 18, 7]
दु:खको दिलामा ईश्वर बोल्दछ
True


In [16]:
lines = txt.splitlines()
for line in lines:
    print(encode(line))

[1753, 1742, 1754]
[1577, 27, 1760, 1165, 193, 1301, 1760, 1729, 681, 1760, 1729, 681, 1760, 1730, 1277]
[1, 1305, 15, 1760, 933, 1465, 593, 1357, 1760, 20, 8, 27, 1760, 1733, 633, 1277, 1752]
[237, 721, 1760, 909, 2, 177, 1760, 1237, 20, 1760, 1025, 27, 149, 30, 1752]
[505, 985, 1760, 1730, 1425, 1756, 809, 27, 1760, 23, 457, 1277]
आँ
[281, 25, 1760, 1733, 453, 1249, 1365, 1760, 3, 20, 1760, 1029, 32, 1752]
[1735, 1, 1760, 1121, 981, 909, 1205, 1760, 265, 28, 1760, 2, 1513, 1277]
[465, 29, 20, 177, 1760, 1731, 813, 1557, 32, 1752]
[1753, 1743, 1754]
[1577, 27, 1759, 1760, 1577, 27, 1760, 817, 1760, 513, 805, 1293, 1760, 373, 17, 1293]
[1557, 26, 1757, 1760, 1577, 1760, 32, 25, 26, 1760, 989, 1465, 645, 27, 1752]
[465, 29, 20, 21, 1760, 17, 1205, 1760, 1121, 369, 27, 1760, 21, 17, 1293, 1757]
[151, 1033, 27, 1577, 177, 1760, 865, 27, 865, 27, 1752]
उँ
[1760, 9, 1333, 28, 27, 1760, 529, 1345, 1760, 23, 721, 7, 1759, 1760, 23, 1129, 1293, 1752]
[1, 27, 1357, 1760, 1729, 1509, 857, 27, 17